In [12]:
# Import dependencies
from bs4 import BeautifulSoup
import requests
import selenium
from splinter import Browser
import pandas as pd
import re
import time

In [13]:
executable_path = {'executable_path': r'C:\webdriver\chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)

In [15]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [16]:
# retrieve title and paragraph for first news title on NASA site
news_title = soup.find("li", class_="slide").find("div", class_="content_title").text
news_p = soup.find("li", class_="slide").find("div", class_="article_teaser_body").text

In [17]:
#JPL Mars Space Images - Featured Image
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [18]:
html = browser.html
soup2 = BeautifulSoup(html, 'html.parser')

In [19]:
# retrieve the text from the style which includes background image url
# jpl_carousel = soup2.find('article', class_='carousel_item')
div_style = soup2.find('article')['style']

In [20]:
# extract image location from style and combine with jpl index url
short_url = div_style.replace("background-image: url('","").replace("');","")
featured_image_url = "https://www.jpl.nasa.gov"+short_url

In [21]:
# Mars Weather
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)
time.sleep(5)

In [22]:
html = browser.html
soup3 = BeautifulSoup(html, 'html.parser')

In [23]:
# retrieve Mars weather tweet
mars_tweet = soup3.find('div', class_='css-1dbjc4n')
weather_tweets = mars_tweet(text=re.compile(r' sol'))
mars_weather = weather_tweets[0]

In [24]:
# Mars Facts
url4 = 'https://space-facts.com/mars/'
browser.visit(url4)

In [25]:
html = browser.html
soup4 = BeautifulSoup(html, 'html.parser')

In [26]:
#retrieve Mars Facts
table = soup4.find('table', class_='tablepress')
table = pd.read_html(url4)
mars_df = table[0]
html_table = mars_df.to_html()
html_table=html_table.replace('\n', '')

In [27]:
# Mars Hemispheres
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url5)

In [28]:
# use a loop to loop through the 4 hemispheres, click through to their page and pull high res image

html = browser.html
soup5 = BeautifulSoup(html, 'html.parser')

hemi_list = []

hemispheres = soup5.find_all('div', class_="item")
for hemi in hemispheres:
    link = hemi.a['href']
    link_long = "https://astrogeology.usgs.gov"+link
    browser.visit(link_long)
    html2 = browser.html
    soup6 = BeautifulSoup(html2, 'html.parser')
    raw_title = soup6.find('title').text
    title = raw_title.replace("Enhanced | USGS Astrogeology Science Center","").replace("e ","e")
    hemi_dict = {}
    hemi_dict.update({'title': title})
    short_img_url = soup6.find('img', class_="wide-image")["src"]
    img_url = "https://astrogeology.usgs.gov"+short_img_url
    hemi_dict.update({'img_url': img_url})
    hemi_list.append(hemi_dict)    

In [29]:
browser.quit()